In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/share/hel/home/mathias/Download/performer-pytorch/')
sys.path.append('/share/hel/home/mathias/Download/fast-transformers/')

In [4]:
from pretty_midi import PrettyMIDI
import torch
import os
from CIA.getters import get_handler
import time
from CIA.getters import get_data_processing, get_model, get_positional_embedding, get_sos_embedding
import importlib
from CIA.positional_embeddings.positional_embedding import PositionalEmbedding
from torch.nn.parallel import DistributedDataParallel

ModuleNotFoundError: No module named 'DatasetManager'

In [3]:
from CIA.utils import get_free_port


GPU = 3
rank = GPU
world_size = 1
rank = 0
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = str(get_free_port())
if torch.cuda.is_available() and GPU in list(range(4)):
    torch.distributed.init_process_group(backend="nccl", world_size=world_size, rank=rank)
    torch.cuda.set_device(rank)
    device_ids = [rank]
    output_device = rank
    device = f"cuda:{rank}"
else:
    # cpu case
    torch.distributed.init_process_group(backend="gloo", world_size=world_size, rank=rank)
    device_ids = None
    output_device = None
    device = "cpu"


In [6]:

# config =  importlib.import_module('CIA.configs.piarceiverRw').config
config =  importlib.import_module('CIA.configs.piarceiverStack').config
# config = importlib.import_module('.config', f'{model_dir.replace("/", ".")}').config
model_dir = 'models/piano_event_performer_2021-10-01_16:03:06'
load = True
overfitted = True

# === Model ====
dataloader_generator, data_processor = get_data_processing(
    dataset=config["dataset"],
    dataloader_generator_kwargs=config["dataloader_generator_kwargs"],
    data_processor_kwargs=config["data_processor_kwargs"],
)
# positional embedding
positional_embedding: PositionalEmbedding = get_positional_embedding(
    dataloader_generator=dataloader_generator,
    data_processor=data_processor,
    positional_embedding_dict=config["positional_embedding_dict"],
)

# sos embedding
sos_embedding = get_sos_embedding(
    dataloader_generator=dataloader_generator,
    sos_embedding_dict=config["sos_embedding_dict"],
)

# TODO: don't override config
# config = importlib.import_module('.config', f'{model_dir.replace("/", ".")}').config
model = get_model(
    data_processor=data_processor,
    dataloader_generator=dataloader_generator,
    positional_embedding=positional_embedding,
    sos_embedding=sos_embedding,
    model_kwargs=config["model_kwargs"],
    # model_kwargs=config["decoder_kwargs"]
)

model.to(device)
model = DistributedDataParallel(
    module=model, device_ids=device_ids, output_device=output_device
)

model_handler = get_handler(
    model=model,
    model_dir=model_dir,
    dataloader_generator=dataloader_generator,
)

if load:
    if overfitted:
        model_handler.load(early_stopped=False, device=device)
    else:
        model_handler.load(early_stopped=True, device=device)


model = model.to(device)
data_processor = data_processor.to(device)

# model_dir = "lolo"
(_, generator_val, _) = dataloader_generator.dataloaders(
    batch_size=1, shuffle_val=True
)
null_superconditioning = [1, 1.5, 2, 2.5]

id_ = 2
for i in range(id_):
    original_x = next(generator_val)["x"]
    _, metadata_dict = data_processor.preprocess(
        original_x,
        num_events_inpainted=100,
        training=False,
        non_conditioned_examples=(null_superconditioning is not None),
    )
x = metadata_dict["original_sequence"]



# model_handler = get_handler(
#     model=model,
#     model_dir=model_dir,
#     dataloader_generator=dataloader_generator,
# )
############################################################
# inpainting

# if load:
#     if overfitted:
#         model_handler.load(early_stopped=False, device=device)
#     else:
#         model_handler.load(early_stopped=True, device=device)

start_time = time.time()
(
    x_gen,
    _,
    decoding_end,
    num_event_generated,
    _,
) = model_handler.inpaint_non_optimized_superconditioning(
    x=x.clone(),
    metadata_dict=metadata_dict,
    temperature=1.0,
    top_p=0.95,
    top_k=0,
    regenerate_first_ts=False,
    null_superconditioning=null_superconditioning,
)
end_time = time.time()
############################################################


KeyError: 'offset_beginning'

In [5]:
PrettyMIDI
x_gen.shape

torch.Size([4, 1024, 4])

In [12]:
x.shape

torch.Size([1, 1024, 4])

In [23]:
dataloader_generator.write(x.cpu()[0], 'out/orig.mid')

KeyError: 92

In [8]:
generator_val

<generator object PianoDataloaderGenerator.dataloaders.<locals>._build_dataloader at 0x7efca399d690>